In [31]:
import pandas as pd


In [32]:
movies = pd.read_csv('ml-latest/movies.csv')
display(movies.head())
display(movies.tail())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,movieId,title,genres
86532,288967,State of Siege: Temple Attack (2021),Action|Drama
86533,288971,Ouija Japan (2021),Action|Horror
86534,288975,The Men Who Made the Movies: Howard Hawks (1973),Documentary
86535,288977,Skinford: Death Sentence (2023),Crime|Thriller
86536,288983,UNZIPPED: An Autopsy of American Inequality (2...,Documentary


In [33]:
tags = pd.read_csv('ml-latest/tags.csv')
display(tags.head())
display(tags.tail())

,userId,movieId,tag,timestamp
0,10,260,good vs evil,1430666558
1,10,260,Harrison Ford,1430666505
2,10,260,sci-fi,1430666538
3,14,1221,Al Pacino,1311600756
4,14,1221,mafia,1311600746


,userId,movieId,tag,timestamp
2328310,330923,176599,politically correct,1507547491
2328311,330933,3317,coming of age,1351279384
2328312,330933,3317,sexuality,1351279389
2328313,330947,5782,Not Luc Besson,1154110902
2328314,330947,44191,comic book,1154271525


In [34]:
ratings = pd.read_csv('ml-latest/ratings.csv')
display(ratings.head())
display(ratings.tail())

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119


,userId,movieId,rating,timestamp
33832157,330975,8340,2.0,1091583256
33832158,330975,8493,2.5,1091585709
33832159,330975,8622,4.0,1091581777
33832160,330975,8665,3.0,1091581765
33832161,330975,8783,2.5,1091581799


In [35]:
tags['tag'] = tags['tag'].astype(str)
tags_groupped_by_movie = tags.groupby('movieId')['tag'].apply(lambda tags: ' '.join(tags)).reset_index()

tags_groupped_by_movie.tail()

,movieId,tag
53447,288765,post-apocalyptic survival tw suicide apocalyps...
53448,288779,Don Camillo Series
53449,288849,addiction animation short film
53450,288937,anime
53451,288955,catalan Costa brava flatmate Millennials slow ...


In [36]:
movies_with_tags = movies.merge(tags_groupped_by_movie, on='movieId', how='left')
movies_with_tags[movies_with_tags['tag'].isna()]

,movieId,title,genres,tag
50,51,Guardian Angel (1994),Action|Drama|Thriller,NaN
55,56,Kids of the Round Table (1995),Adventure|Children|Comedy|Fantasy,NaN
83,84,Last Summer in the Hamptons (1995),Comedy|Drama,NaN
89,90,The Journey of August King (1995),Drama,NaN
106,108,Catwalk (1996),Documentary,NaN
...,...,...,...,...
86532,288967,State of Siege: Temple Attack (2021),Action|Drama,NaN
86533,288971,Ouija Japan (2021),Action|Horror,NaN
86534,288975,The Men Who Made the Movies: Howard Hawks (1973),Documentary,NaN
86535,288977,Skinford: Death Sentence (2023),Crime|Thriller,NaN


In [37]:
movies_with_tags.count()

movieId    86537
title      86537
genres     86537
tag        53452
dtype: int64

In [38]:
grouped_rating = ratings.groupby('movieId')['rating']
average_ratings_median = grouped_rating.median().reset_index()
average_ratings_var = grouped_rating.var().reset_index()

movies_total = movies_with_tags.merge(average_ratings_median, on='movieId', how='left')
movies_total = movies_total.merge(average_ratings_var, on='movieId', how='left')
movies_total.rename(columns={'rating_x': 'median_rating', 'rating_y': 'variance_rating'}, inplace=True)
movies_total['genres'] = movies_total['genres'].str.replace('|', ' ').replace('  ', ' ')
movies_total['genres_tag'] = movies_total['genres'] + ' ' + movies_total['tag'].fillna('')
movies_total = movies_total[['movieId', 'title', 'genres_tag', 'median_rating', 'variance_rating']]
movies_total.count()

movieId            86537
title              86537
genres_tag         86537
median_rating      83239
variance_rating    65323
dtype: int64

In [39]:
movies_total.dropna(subset=['variance_rating'], inplace=True)
movies_total.count()

movieId            65323
title              65323
genres_tag         65323
median_rating      65323
variance_rating    65323
dtype: int64

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(movies_total['genres_tag'])
X_train_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1607112 stored elements and shape (65323, 51323)>

In [41]:
from scipy.sparse import hstack

train_data = hstack((X_train_tfidf, movies_total[['variance_rating']]))
train_data

<COOrdinate sparse matrix of dtype 'float64'
	with 1670327 stored elements and shape (65323, 51324)>

In [42]:
pd.DataFrame.sparse.from_spmatrix(train_data, index=movies_total.index).head()

,0,1,2,3,4,5,6,7,8,9,...,51314,51315,51316,51317,51318,51319,51320,51321,51322,51323
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.863236
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.919146
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.032378
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.238456
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.999141


In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data, movies_total['median_rating'], test_size=0.2, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((52258, 51324), (52258,), (13065, 51324), (13065,))

In [52]:
from sklearn.linear_model import Ridge

model = Ridge(alpha=1.0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

np.float64(1.3209439372589462)

In [46]:
from sklearn.metrics import root_mean_squared_error

root_mean_squared_error(y_test, y_pred)

np.float64(0.6475878732058049)

In [47]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_test, y_pred)

np.float64(0.47943298853816124)